This time around you will solve the MDP defined in the last part. You will get that ice cream! First, we need a bit more learnings:

# Discounted Future Reward

Why do we need the discount factor $\gamma$? The total reward that our agent will receive from the current time step t to the end of the task can be defined as:

$$R_t = r_t + r_{t + 1} + \ldots + r_n$$

That looks ok, but let's not forget that our environment is stochastic (the supermarket might close any time now). The discount factor allows us to value short-term reward more than long-term ones, we can use it as:

$$R_t = R_t + \gamma r_{t+1} + \ldots + \gamma^{n - t} r_n$$

Our agent would perform great if he chooses the action that maximizes the (discounted) future reward at every step.

# Value function

It would be great to know how "good" a given state $s$ is. Something to tell us - no matter the state you're in if you transition to state $s$ your total reward will be $x$, word! If you start from $s$ and follow policy $\pi$. That would spare us from revisiting same states over and over again. The **value function** does this for us. It depends on the state we're in $s$ and the policy $\pi$ our agent is following. It is given by:

$$V^{\pi}(s) = \mathbb{E}(\sum_{t \geq 0}\gamma^t r_t) \quad \forall s \in \mathbb{S}$$

There exists an **optimal value function** that has the highest value for all states. It is given by:

$$V^*(s) = \max_{\pi}V^{\pi}(s) \quad \forall s \in \mathbb{S}$$

# Q function

Yet, your agent can't control what state he ends up in, directly. He can influence it by choosing some action $a$.  Let's introduce another function that accepts state and action as parameters and returns the expected total reward - the Q function. More formally, the function $Q^{\pi}(s, a)$ gives the expected return when starting in $s$, performing $a$ and following $\pi$.

Again, we can define the optimal Q-function $Q^*(s, a)$ that gives the expected total reward for our agent when starting at $s$ and picks action $a$. That is, the optimal Q-function tells our agent how good of a choice is picking $a$ when at state $s$.

There is a relationship between the two optimal functions $V^*$ and $Q^*$. Since $V^*$ is the highest expected total reward when starting at $s$, it will be 